In [1]:
import openai
import csv
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np
import sys
import re
import time
import glob
import os


### Convert Food data into something readable for the machine

In [5]:
# Function to extract data from the XML and write to files
def process_xml_to_files(xml_root, file1_path, file2_path):
    with open(file1_path, 'w', newline='', encoding='utf-8') as file1, \
         open(file2_path, 'w', newline='', encoding='utf-8') as file2:

        writer1 = csv.writer(file1, delimiter='|')
        writer2 = csv.writer(file2, delimiter='|')

        for document in xml_root.findall('.//document'):
            recipe_id = document.find('id').text
            full_text = document.find(".//infon[@key='full_text']").text
            writer1.writerow([recipe_id, full_text])
            for annotation in document.findall('.//annotation'):
                annotated_text = annotation.find('text').text
                semantic_tags = annotation.find(".//infon[@key='semantic_tags']").text
                writer2.writerow([recipe_id, annotated_text, semantic_tags])

# Load the XML file
tree = ET.parse('CafeteriaCorpus/CafeteriaSA/CafeteriaSA_SNOMEDCT.xml')
root = tree.getroot()

# File paths for the output files
file1_path = 'CafeteriaCorpus/CafeteriaSA/CafeteriaSA_SNOMEDCT_reformatted_recipes.csv'
file2_path = 'CafeteriaCorpus/CafeteriaSA/CafeteriaSA_SNOMEDCT_reformatted_annotations.csv'

# Process the XML and write to the files
process_xml_to_files(root, file1_path, file2_path)

### Start getting ChatGPT annotations for food

In [ ]:
#datoteka = "CafeteriaCorpus/CafeteriaSA/CafeteriaSA_FOODON_reformatted_recipes.csv"
datoteka = "Biocreative_dataset/disease/TestSet.DNorm.csv"
# read the file
#ime_rezultatov = "CafeteriaSA_foodon_response_GPT4.csv"
ime_rezultatov = "TestSet.DNorm.chatGPT4Results.csv"


df = pd.read_csv(datoteka)
# recipe_id|text
#start = "4recipe779"
go = True
count_limit = 0
data_frame_new = pd.read_csv("results_chatGPT-4/disease/"+ime_rezultatov)
for index,row in df.iterrows():
    title_and_abstract = row.title+row.abstract
    #print(title_and_abstract)
    print(row.pubmed_id)
    if go:
        try:
            chat = openai.ChatCompletion.create(
                model="gpt-4",
                messages=[
                    {"role": "system", "content": "You are a NER tool"},
                    #{"role": "assistant", "content": "Extract only disease entities and find their correct MESH ids,store them in csv format delimiter is | in \"Famotidine is a histamine H2-receptor antagonist used in inpatient settings for prevention of stress ulcers and is showing increasing popularity because of its low cost. Although all of the currently available H2-receptor antagonists have shown the propensity to cause delirium, only two previously reported cases have been associated with famotidine. The authors report on six cases of famotidine-associated delirium in hospitalized patients who cleared completely upon removal of famotidine. The pharmacokinetics of famotidine are reviewed, with no change in its metabolism in the elderly population seen. The implications of using famotidine in elderly persons are discussed.\""},
                    {"role": "assistant", "content": "Extract only disease entities and find their correct MESH ids,display the response only in this format name_of_disease|MESH_id without headers and additional text.Text is:\""+title_and_abstract+"\""},
                ]
            )
            reply = chat.choices[0].message
            new_row = [row.pubmed_id,reply.content]
            data_frame_new = data_frame_new.append(pd.Series(new_row,index=["pubmed_id","result"]),ignore_index=True)
            count_limit = count_limit + 1
        except:
            print("EXCEPTION HAPPENED")
            data_frame_new.to_csv('results_chatGPT-4/disease/'+ime_rezultatov,index=False)
            time.sleep(60)
            count_limit = 0
            chat = openai.ChatCompletion.create(
                model="gpt-4",
                messages=[
                    {"role": "system", "content": "You are a NER tool"},
                    #{"role": "assistant", "content": "Extract only disease entities and find their correct MESH ids,store them in csv format delimiter is | in \"Famotidine is a histamine H2-receptor antagonist used in inpatient settings for prevention of stress ulcers and is showing increasing popularity because of its low cost. Although all of the currently available H2-receptor antagonists have shown the propensity to cause delirium, only two previously reported cases have been associated with famotidine. The authors report on six cases of famotidine-associated delirium in hospitalized patients who cleared completely upon removal of famotidine. The pharmacokinetics of famotidine are reviewed, with no change in its metabolism in the elderly population seen. The implications of using famotidine in elderly persons are discussed.\""},
                    #{"role": "assistant", "content": "Extract only food entities and find their correct FOODON ids,display the response only in this format name_of_food|FOODON_id without headers and additional text.Text is:\""+row.text.strip()+"\""},
                    {"role": "assistant", "content": "Extract only disease entities and find their correct MESH ids,display the response only in this format name_of_disease|MESH_id without headers and additional text.Text is:\""+title_and_abstract+"\""},
                ]
            )
            reply = chat.choices[0].message
            new_row = [row.pubmed_id,reply.content]
            data_frame_new = data_frame_new.append(pd.Series(new_row,index=["pubmed_id","result"]),ignore_index=True)
            count_limit = count_limit + 1
    if count_limit == 450:
        data_frame_new.to_csv('results_chatGPT-4/disease/'+ime_rezultatov,index=False)
        time.sleep(60)
        count_limit = 0
    
    #if str(row.recipe_id) == str(start):
    #    go = True
if count_limit < 450 and count_limit > 0:
    data_frame_new.to_csv('results_chatGPT-4/disease/'+ime_rezultatov,index=False)
    count_limit = 0

# CHATGPT-3.5
___

In [122]:
# --- Results chatGPT ---
cafeteria_SA_foodon_results = pd.read_csv("results_chatGPT-3_5/food/CafeteriaSA_FOODON_response.csv",delimiter=',')
cafeteria_FCD_foodon_results = pd.read_csv("results_chatGPT-3_5/food/CafeteriaFCD_foodon_response.csv",delimiter=',')
cafeteria_SA_snomedct_results = pd.read_csv("results_chatGPT-3_5/food/CafeteriaSA_SNOMEDCT_response.csv",delimiter=',')
cafeteria_FCD_snomedct_results = pd.read_csv("results_chatGPT-3_5/food/CafeteriaFCD_snomedct_response.csv",delimiter=',') 


# test set
cafeteria_SA_foodon = pd.read_csv("CafeteriaCorpus/CafeteriaSA/CafeteriaSA_FOODON_reformatted_annotations.csv",delimiter='|')
cafeteria_FCD_foodon = pd.read_csv("CafeteriaCorpus/CafeteriaFCD/CafeteriaFCD_foodon_reformatted_annotations.csv",delimiter='|')
cafeteria_SA_snomedct = pd.read_csv("CafeteriaCorpus/CafeteriaSA/CafeteriaSA_FOODON_reformatted_annotations.csv",delimiter='|')
cafeteria_FCD_snomedct = pd.read_csv("CafeteriaCorpus/CafeteriaFCD/CafeteriaFCD_foodon_reformatted_annotations.csv",delimiter='|')

# we get the first columns
first_column_cafeteria_SA_foodon = set(cafeteria_SA_foodon.iloc[:,0])
first_column_cafeteria_FCD_foodon = set(cafeteria_FCD_foodon.iloc[:,0])
first_column_cafeteria_SA_snomedct = set(cafeteria_SA_snomedct.iloc[:,0])
first_column_cafeteria_FCD_snomedct = set(cafeteria_FCD_snomedct.iloc[:,0])

### Data preprocessing chatgpt-3.5 SA_FOODON

In [2]:
# --- Results chatGPT ---
cafeteria_SA_foodon_results = pd.read_csv("results_chatGPT-3_5/food/CafeteriaSA_FOODON_response.csv",delimiter=',')
cafeteria_SA_foodon = pd.read_csv("CafeteriaCorpus/CafeteriaSA/CafeteriaSA_FOODON_reformatted_annotations.csv",delimiter='|')
first_column_cafeteria_SA_foodon = set(cafeteria_SA_foodon.iloc[:,0])


# we prepare the chatGPT data so that we can easily parse it
dictionary_of_chatGPT_results = {}
dictionary_of_chatGPT_results_without_MESH = {}
for index,row in cafeteria_SA_foodon_results.iterrows():
    chatGPT_annotations = set()
    chatGPT_annotations_2 = set()
    all_annotations = row["result"]
    for chatGPTanno in all_annotations.split('\n'):
        if len(chatGPTanno) > 0:
                chatGPT_annotations.add((chatGPTanno.strip().split('|')[0],chatGPTanno.strip().split('|')[1]))
                chatGPT_annotations_2.add(chatGPTanno.strip().split('|')[0])
    dictionary_of_chatGPT_results[str(row[0])] = chatGPT_annotations
    dictionary_of_chatGPT_results_without_MESH[str(row[0])] = chatGPT_annotations_2
# test set
correct_entity = 0
incorrect_entity = 0
correct_entity_and_MESH = 0
incorrect_entity_and_MESH = 0
all_correct_entities = 0


### ------------------------------------------------------------------------------------------------------------------------

# cafeteria_SA_foodon
dictionary_of_results = {}
dictionary_of_results_without_MESH = {}
for article_identifier in first_column_cafeteria_SA_foodon:
    filtered_cafeteria_SA_foodon = cafeteria_SA_foodon[cafeteria_SA_foodon.iloc[:, 0] == article_identifier]
    set_for_both = {}
    set_for_names = set()
    for index,row in filtered_cafeteria_SA_foodon.iterrows():
        name = str(row[1])
        identifiers = str(row[2]).split(';')
        list_of_possible_identifiers = set()
        for identity in identifiers:
            list_of_possible_identifiers.add(identity.split('_')[1])
        # 0 is the actually name and 1 is a set of possible identifiers
        set_for_both[name] = list_of_possible_identifiers
        set_for_names.add(name)
    dictionary_of_results[article_identifier] = set_for_both
    dictionary_of_results_without_MESH[article_identifier] = set_for_names
    all_correct_entities += len(set_for_names)

#print(dictionary_of_chatGPT_results)
#print(dictionary_of_chatGPT_results_without_MESH)
#print(dictionary_of_results)
#print(dictionary_of_results_without_MESH)

print(len(dictionary_of_chatGPT_results_without_MESH))
print(len(dictionary_of_results_without_MESH))

print(len(dictionary_of_chatGPT_results))
print(len(dictionary_of_results))


# Some of them are missing so we need to add them with empty array:
what_is_missing = []
for element in dictionary_of_chatGPT_results_without_MESH:
    does_it_exist = False
    for element2 in dictionary_of_results_without_MESH:
        if str(element) == str(element2):
            does_it_exist = True
    if not does_it_exist:
         what_is_missing.append(element)

         
for element in what_is_missing:
    dictionary_of_results[element] = {}
    dictionary_of_results_without_MESH[element] = set()

print(len(dictionary_of_chatGPT_results))
print(len(dictionary_of_results))

path_to_results_file_without_mesh = "basic_occurences_results/chatgpt-3.5/food/chatGPT_food_statistics_without_FOODON_SA_FOODON.txt"
path_to_results_file_with_mesh = "basic_occurences_results/chatgpt-3.5/food/chatGPT_food_statistics_with_FOODON_SA_FOODON.txt"
path_to_confusion_matrix_pure = "confusion_matrix_results_CHATGPT/chatgpt-3.5/food/confusion_matrix_food_pure_entities_SA_FOODON.csv"
path_to_confusion_matrix_partial_lapover = "confusion_matrix_results_CHATGPT/chatgpt-3.5/food/confusion_matrix_food_entities_test_contains_SA_FOODON.csv"
path_to_confusion_matrix_partial_full_lapover = "confusion_matrix_results_CHATGPT/chatgpt-3.5/food/confusion_matrix_food_entities_test_contains_and_chatgpt_contains_SA_FOODON.csv"


501
485
501
485
501
501


### Data preprocessing chatgpt-3.5 SA_SNOMEDCT

In [17]:
cafeteria_SA_snomedct_results = pd.read_csv("results_chatGPT-3_5/food/CafeteriaSA_SNOMEDCT_response.csv",delimiter=',')
cafeteria_SA_snomedct = pd.read_csv("CafeteriaCorpus/CafeteriaSA/CafeteriaSA_SNOMEDCT_reformatted_annotations.csv",delimiter='|')
first_column_cafeteria_SA_snomedct = set(cafeteria_SA_snomedct.iloc[:,0])

#print(article_results_chatGPT)

# we prepare the chatGPT data so that we can easily parse it
dictionary_of_chatGPT_results = {}
dictionary_of_chatGPT_results_without_MESH = {}
for index,row in cafeteria_SA_snomedct_results.iterrows():
    chatGPT_annotations = set()
    chatGPT_annotations_2 = set()
    all_annotations = row["result"]
    for chatGPTanno in all_annotations.split('\n'):
        if len(chatGPTanno) > 0:
                chatGPT_annotations.add((chatGPTanno.strip().split('|')[0],chatGPTanno.strip().split('|')[1]))
                chatGPT_annotations_2.add(chatGPTanno.strip().split('|')[0])
    dictionary_of_chatGPT_results[str(row[0])] = chatGPT_annotations
    dictionary_of_chatGPT_results_without_MESH[str(row[0])] = chatGPT_annotations_2
# test set
correct_entity = 0
incorrect_entity = 0
correct_entity_and_MESH = 0
incorrect_entity_and_MESH = 0
all_correct_entities = 0


### ------------------------------------------------------------------------------------------------------------------------

# cafeteria_SA_foodon
dictionary_of_results = {}
dictionary_of_results_without_MESH = {}
for article_identifier in first_column_cafeteria_SA_snomedct:
    filtered_cafeteria_SA_foodon = cafeteria_SA_snomedct[cafeteria_SA_snomedct.iloc[:, 0] == article_identifier]
    set_for_both = {}
    set_for_names = set()
    for index,row in filtered_cafeteria_SA_foodon.iterrows():
        name = str(row[1])
        identifiers = str(row[2]).split(';')
        list_of_possible_identifiers = set()
        for identity in identifiers:
            divide_identifiers = identity.split('/')
            get_correct = divide_identifiers[len(divide_identifiers)-1]
            list_of_possible_identifiers.add(get_correct)
        # 0 is the actually name and 1 is a set of possible identifiers
        set_for_both[name] = list_of_possible_identifiers
        set_for_names.add(name)
    dictionary_of_results[article_identifier] = set_for_both
    dictionary_of_results_without_MESH[article_identifier] = set_for_names
    all_correct_entities += len(set_for_names)

#print(dictionary_of_chatGPT_results)
#print(dictionary_of_chatGPT_results_without_MESH)
#print(dictionary_of_results)
#print(dictionary_of_results_without_MESH)
print(len(dictionary_of_chatGPT_results_without_MESH))
print(len(dictionary_of_results_without_MESH))

print(len(dictionary_of_chatGPT_results))
print(len(dictionary_of_results))


# Some of them are missing so we need to add them with empty array:
what_is_missing = []
for element in dictionary_of_chatGPT_results_without_MESH:
    does_it_exist = False
    for element2 in dictionary_of_results_without_MESH:
        if str(element) == str(element2):
            does_it_exist = True
    if not does_it_exist:
         what_is_missing.append(element)

         
for element in what_is_missing:
    dictionary_of_results[element] = {}
    dictionary_of_results_without_MESH[element] = set()

print(len(dictionary_of_chatGPT_results))
print(len(dictionary_of_results))


path_to_results_file_without_mesh = "basic_occurences_results/chatgpt-3.5/food/chatGPT_food_statistics_without_SNOMEDCT_SA_SNOMEDCT.txt"
path_to_results_file_with_mesh = "basic_occurences_results/chatgpt-3.5/food/chatGPT_food_statistics_with_SNOMEDCT_SA_SNOMEDCT.txt"
path_to_confusion_matrix_pure = "confusion_matrix_results_CHATGPT/chatgpt-3.5/food/confusion_matrix_food_pure_entities_SA_SNOMEDCT.csv"
path_to_confusion_matrix_partial_lapover = "confusion_matrix_results_CHATGPT/chatgpt-3.5/food/confusion_matrix_food_entities_test_contains_SA_SNOMEDCT.csv"
path_to_confusion_matrix_partial_full_lapover = "confusion_matrix_results_CHATGPT/chatgpt-3.5/food/confusion_matrix_food_entities_test_contains_and_chatgpt_contains_SA_SNOMEDCT.csv"


501
476
501
476
501
501


### Data preprocessing chatgpt-3.5 FCD_FOODON

In [32]:
cafeteria_FCD_foodon_results = pd.read_csv("results_chatGPT-3_5/food/CafeteriaFCD_foodon_response.csv",delimiter=',')
cafeteria_FCD_foodon = pd.read_csv("CafeteriaCorpus/CafeteriaFCD/CafeteriaFCD_foodon_reformatted_annotations.csv",delimiter='|')
first_column_cafeteria_FCD_foodon = set(cafeteria_FCD_foodon.iloc[:,0])

#print(article_results_chatGPT)

# we prepare the chatGPT data so that we can easily parse it
dictionary_of_chatGPT_results = {}
dictionary_of_chatGPT_results_without_MESH = {}
for index,row in cafeteria_FCD_foodon_results.iterrows():
    chatGPT_annotations = set()
    chatGPT_annotations_2 = set()
    all_annotations = row["result"]
    for chatGPTanno in str(all_annotations).split('\n'):
        if len(chatGPTanno) > 0 and str(chatGPTanno) != 'nan':
                print(chatGPTanno)
                chatGPT_annotations.add((chatGPTanno.strip().split('|')[0],chatGPTanno.strip().split('|')[1]))
                chatGPT_annotations_2.add(chatGPTanno.strip().split('|')[0])
    dictionary_of_chatGPT_results[str(row[0])] = chatGPT_annotations
    dictionary_of_chatGPT_results_without_MESH[str(row[0])] = chatGPT_annotations_2
# test set
correct_entity = 0
incorrect_entity = 0
correct_entity_and_MESH = 0
incorrect_entity_and_MESH = 0
all_correct_entities = 0


### ------------------------------------------------------------------------------------------------------------------------

# cafeteria_SA_foodon
dictionary_of_results = {}
dictionary_of_results_without_MESH = {}
for article_identifier in first_column_cafeteria_FCD_foodon:
    filtered_cafeteria_SA_foodon = cafeteria_FCD_foodon[cafeteria_FCD_foodon.iloc[:, 0] == article_identifier]
    set_for_both = {}
    set_for_names = set()
    for index,row in filtered_cafeteria_SA_foodon.iterrows():
        name = str(row[1])
        identifiers = str(row[2]).split(';')
        list_of_possible_identifiers = set()
        for identity in identifiers:
            list_of_possible_identifiers.add(identity.split('_')[1])
        # 0 is the actually name and 1 is a set of possible identifiers
        set_for_both[name] = list_of_possible_identifiers
        set_for_names.add(name)
    dictionary_of_results[article_identifier] = set_for_both
    dictionary_of_results_without_MESH[article_identifier] = set_for_names
    all_correct_entities += len(set_for_names)

#print(dictionary_of_chatGPT_results)
#print(dictionary_of_chatGPT_results_without_MESH)
#print(dictionary_of_results)
#print(dictionary_of_results_without_MESH)

print(len(dictionary_of_chatGPT_results_without_MESH))
print(len(dictionary_of_results_without_MESH))

print(len(dictionary_of_chatGPT_results))
print(len(dictionary_of_results))


# Some of them are missing so we need to add them with empty array:
what_is_missing = []
for element in dictionary_of_chatGPT_results_without_MESH:
    does_it_exist = False
    for element2 in dictionary_of_results_without_MESH:
        if str(element) == str(element2):
            does_it_exist = True
    if not does_it_exist:
         what_is_missing.append(element)

         
for element in what_is_missing:
    dictionary_of_results[element] = {}
    dictionary_of_results_without_MESH[element] = set()

print(len(dictionary_of_chatGPT_results))
print(len(dictionary_of_results))

path_to_results_file_without_mesh = "basic_occurences_results/chatgpt-3.5/food/chatGPT_food_statistics_without_FOODON_FCD_FOODON.txt"
path_to_results_file_with_mesh = "basic_occurences_results/chatgpt-3.5/food/chatGPT_food_statistics_with_FOODON_FCD_FOODON.txt"
path_to_confusion_matrix_pure = "confusion_matrix_results_CHATGPT/chatgpt-3.5/food/confusion_matrix_food_pure_entities_FCD_FOODON.csv"
path_to_confusion_matrix_partial_lapover = "confusion_matrix_results_CHATGPT/chatgpt-3.5/food/confusion_matrix_food_entities_test_contains_FCD_FOODON.csv"
path_to_confusion_matrix_partial_full_lapover = "confusion_matrix_results_CHATGPT/chatgpt-3.5/food/confusion_matrix_food_entities_test_contains_and_chatgpt_contains_FCD_FOODON.csv"


cream cheese|03300621
beef|03402088
olives|03402062
onion|00001878
Worcestershire sauce|03411084
walnuts|03400740
cheese ball|03401249
serving plate|03401748
waxed paper|03402702
artichoke hearts|00001899
mozzarella cheese|01306852
parmesan cheese|01303959
mayonnaise|00001285
almonds|01301019
tortilla chips|01303194
crackers|01300824
sun-dried tomatoes|03402890
garlic|00000677
salt|00001127
bowl|00001001
garbanzo beans|00003603
tahini|00003682
lemon juice|03422187
honey|00001613
water|00005896
olive oil|00001370
pita breads|00001048
rosemary|00003320
hummus|00001021
chicken|03310602
picante sauce|03310599
ground cumin|03210312
vegetable oil|00003220
corn tortilla|00002531
cheese|04250216
cream cheese|03480122
dry ranch salad dressing mix|03310798
mayonnaise|04050508
milk|03410001
cream cheese|03310337
Cheddar cheese|03200710
almonds|03460019
rice|00001961
lamb|00002247
allspice|03431612
salt|00013588
pepper|00001192
garlic|03431434
lemon juice|03431461
olives|00014173
Grape leaves|0002

### Data preprocessing chatgpt-3.5 FCD_SNOMEDCT

In [49]:
cafeteria_FCD_snomedct_results = pd.read_csv("results_chatGPT-3_5/food/CafeteriaFCD_snomedct_response.csv",delimiter=',')
cafeteria_FCD_snomedct = pd.read_csv("CafeteriaCorpus/CafeteriaFCD/CafeteriaFCD_snomedct_reformatted_annotations.csv",delimiter='|')
first_column_cafeteria_FCD_snomedct = set(cafeteria_FCD_snomedct.iloc[:,0])
#print(article_results_chatGPT)

# we prepare the chatGPT data so that we can easily parse it
dictionary_of_chatGPT_results = {}
dictionary_of_chatGPT_results_without_MESH = {}
for index,row in cafeteria_FCD_snomedct_results.iterrows():
    chatGPT_annotations = set()
    chatGPT_annotations_2 = set()
    all_annotations = row["result"]
    for chatGPTanno in str(all_annotations).split('\n'):
        if len(chatGPTanno) > 0 and chatGPTanno != 'nan':
                print(chatGPTanno)
                chatGPT_annotations.add((chatGPTanno.strip().split('|')[0],chatGPTanno.strip().split('|')[1]))
                chatGPT_annotations_2.add(chatGPTanno.strip().split('|')[0])
    dictionary_of_chatGPT_results[str(row[0])] = chatGPT_annotations
    dictionary_of_chatGPT_results_without_MESH[str(row[0])] = chatGPT_annotations_2
# test set
correct_entity = 0
incorrect_entity = 0
correct_entity_and_MESH = 0
incorrect_entity_and_MESH = 0
all_correct_entities = 0


### ------------------------------------------------------------------------------------------------------------------------

for article_identifier in first_column_cafeteria_FCD_snomedct:
    filtered_cafeteria_SA_foodon = cafeteria_FCD_snomedct[cafeteria_FCD_snomedct.iloc[:, 0] == article_identifier]
    set_for_both = {}
    set_for_names = set()
    for index,row in filtered_cafeteria_SA_foodon.iterrows():
        name = str(row[1])
        identifiers = str(row[2]).split(';')
        list_of_possible_identifiers = set()
        for identity in identifiers:
            divide_identifiers = identity.split('/')
            get_correct = divide_identifiers[len(divide_identifiers)-1]
            list_of_possible_identifiers.add(get_correct)
        # 0 is the actually name and 1 is a set of possible identifiers
        set_for_both[name] = list_of_possible_identifiers
        set_for_names.add(name)
    dictionary_of_results[article_identifier] = set_for_both
    dictionary_of_results_without_MESH[article_identifier] = set_for_names
    all_correct_entities += len(set_for_names)



print(len(dictionary_of_chatGPT_results_without_MESH))
print(len(dictionary_of_results_without_MESH))

print(len(dictionary_of_chatGPT_results))
print(len(dictionary_of_results))


# Some of them are missing so we need to add them with empty array:
what_is_missing = []
for element in dictionary_of_chatGPT_results_without_MESH:
    does_it_exist = False
    for element2 in dictionary_of_results_without_MESH:
        if str(element) == str(element2):
            does_it_exist = True
    if not does_it_exist:
         what_is_missing.append(element)

         
for element in what_is_missing:
    dictionary_of_results[element] = {}
    dictionary_of_results_without_MESH[element] = set()

print(len(dictionary_of_chatGPT_results))
print(len(dictionary_of_results))


path_to_results_file_without_mesh = "basic_occurences_results/chatgpt-3.5/food/chatGPT_food_statistics_without_SNOMEDCT_FCD_SNOMEDCT.txt"
path_to_results_file_with_mesh = "basic_occurences_results/chatgpt-3.5/food/chatGPT_food_statistics_with_SNOMEDCT_FCD_SNOMEDCT.txt"
path_to_confusion_matrix_pure = "confusion_matrix_results_CHATGPT/chatgpt-3.5/food/confusion_matrix_food_pure_entities_FCD_SNOMEDCT.csv"
path_to_confusion_matrix_partial_lapover = "confusion_matrix_results_CHATGPT/chatgpt-3.5/food/confusion_matrix_food_entities_test_contains_FCD_SNOMEDCT.csv"
path_to_confusion_matrix_partial_full_lapover = "confusion_matrix_results_CHATGPT/chatgpt-3.5/food/confusion_matrix_food_entities_test_contains_and_chatgpt_contains_FCD_SNOMEDCT.csv"


cream cheese|36577009
beef|941141000124105
olives|736635009
onion|10200004
Worcestershire sauce|287032009
walnuts|21314007
artichoke hearts|248042001
mozzarella cheese|47101001
parmesan cheese|364570002
mayonnaise|43395002
almonds|118472009
tortilla chips|118454002
crackers|49012006
sun-dried tomatoes|7948
cream cheese|8292
garlic|383155009
provolone cheese|6608
pesto|375254001
garlic|
salt|
bowl|
blender|
food processor|
garbanzo beans|
tahini|
lemon juice|
honey|
water|
serving dish|
olive oil|
oven|
pita breads|
wedges|
rosemary|
hummus|
shredded chicken|289898009
picante sauce|406718008
ground cumin|99999008
vegetable oil|25305005
corn tortilla|282487004
cheese|34095006
spread cream cheese|337765001
meat slice|67244007
pickle piece|74794001
meat|74399006
dry ranch salad dressing mix|470371000119109
mayonnaise|105314001
milk|414035005
cream cheese|129803006
Cheddar cheese|359580002
plastic wrap|
almonds|30014000
rice|xxx
ground lamb|yyy
allspice|zzz
salt|aaa
pepper|bbb
garlic|ccc
le

# Chatgpt-4
___

### Data preprocessing chatgpt-4 SA FOODON 

In [67]:
# --- Results chatGPT ---
cafeteria_SA_foodon_results = pd.read_csv("results_chatgpt-4/food/CafeteriaSA_foodon_response_GPT4.csv",delimiter=',')
cafeteria_SA_foodon = pd.read_csv("CafeteriaCorpus/CafeteriaSA/CafeteriaSA_FOODON_reformatted_annotations.csv",delimiter='|')
first_column_cafeteria_SA_foodon = set(cafeteria_SA_foodon.iloc[:,0])


# we prepare the chatGPT data so that we can easily parse it
dictionary_of_chatGPT_results = {}
dictionary_of_chatGPT_results_without_MESH = {}
for index,row in cafeteria_SA_foodon_results.iterrows():
    chatGPT_annotations = set()
    chatGPT_annotations_2 = set()
    all_annotations = row["result"]
    print(all_annotations)
    for chatGPTanno in str(all_annotations).split('\n'):
        if len(chatGPTanno) > 0 and chatGPTanno != 'nan':
                chatGPT_annotations.add((chatGPTanno.strip().split('|')[0],chatGPTanno.strip().split('|')[1]))
                chatGPT_annotations_2.add(chatGPTanno.strip().split('|')[0])
    dictionary_of_chatGPT_results[str(row[0])] = chatGPT_annotations
    dictionary_of_chatGPT_results_without_MESH[str(row[0])] = chatGPT_annotations_2
# test set
correct_entity = 0
incorrect_entity = 0
correct_entity_and_MESH = 0
incorrect_entity_and_MESH = 0
all_correct_entities = 0


### ------------------------------------------------------------------------------------------------------------------------

# cafeteria_SA_foodon
dictionary_of_results = {}
dictionary_of_results_without_MESH = {}
for article_identifier in first_column_cafeteria_SA_foodon:
    filtered_cafeteria_SA_foodon = cafeteria_SA_foodon[cafeteria_SA_foodon.iloc[:, 0] == article_identifier]
    set_for_both = {}
    set_for_names = set()
    for index,row in filtered_cafeteria_SA_foodon.iterrows():
        name = str(row[1])
        identifiers = str(row[2]).split(';')
        list_of_possible_identifiers = set()
        for identity in identifiers:
            list_of_possible_identifiers.add(identity.split('_')[1])
        # 0 is the actually name and 1 is a set of possible identifiers
        set_for_both[name] = list_of_possible_identifiers
        set_for_names.add(name)
    dictionary_of_results[article_identifier] = set_for_both
    dictionary_of_results_without_MESH[article_identifier] = set_for_names
    all_correct_entities += len(set_for_names)

#print(dictionary_of_chatGPT_results)
#print(dictionary_of_chatGPT_results_without_MESH)
#print(dictionary_of_results)
#print(dictionary_of_results_without_MESH)

print(len(dictionary_of_chatGPT_results_without_MESH))
print(len(dictionary_of_results_without_MESH))

print(len(dictionary_of_chatGPT_results))
print(len(dictionary_of_results))


# Some of them are missing so we need to add them with empty array:
what_is_missing = []
for element in dictionary_of_chatGPT_results_without_MESH:
    does_it_exist = False
    for element2 in dictionary_of_results_without_MESH:
        if str(element) == str(element2):
            does_it_exist = True
    if not does_it_exist:
         what_is_missing.append(element)

         
for element in what_is_missing:
    dictionary_of_results[element] = {}
    dictionary_of_results_without_MESH[element] = set()

print(len(dictionary_of_chatGPT_results))
print(len(dictionary_of_results))


path_to_results_file_without_mesh = "basic_occurences_results/chatgpt-4/food/chatGPT_food_statistics_without_FOODON_SA_FOODON.txt"
path_to_results_file_with_mesh = "basic_occurences_results/chatgpt-4/food/chatGPT_food_statistics_with_FOODON_SA_FOODON.txt"
path_to_confusion_matrix_pure = "confusion_matrix_results_CHATGPT/chatgpt-4/food/confusion_matrix_food_pure_entities_SA_FOODON.csv"
path_to_confusion_matrix_partial_lapover = "confusion_matrix_results_CHATGPT/chatgpt-4/food/confusion_matrix_food_entities_test_contains_SA_FOODON.csv"
path_to_confusion_matrix_partial_full_lapover = "confusion_matrix_results_CHATGPT/chatgpt-4/food/confusion_matrix_food_entities_test_contains_and_chatgpt_contains_SA_FOODON.csv"


gluten-free products|03301856
corn|00002403
rice|00002284
buckwheat|00001173
oat|00001107
amaranth|00001106
teff|00002107
quinoa|00002354
nan
milk|03301505
protein|03303283
phosphorus|03460098
calcium|03461706
energy|03421039
tea|03401017
coffee|03302322
chocolate|03302001
cocoa|03450858
cinnamon|03302109
grape|03432733
pomegranate|03530184
red wine|03303554
berries|03432713
olive oil|03302340
red meat|03301156
processed meat|03310774
ultra-processed foods|03301598
dairy products|03460095
alcohol|03461309
whole grain cereals|03303013
nuts|03303204
vegetables|00002403
fruits|03302003
Curcuma longa|03410488
Nigella sativa|03301204
Zingiber officinale|03307309
Allium sativum|03302057
Crocus sativus|03301526
Piper nigrum|03307186
Capsicum annum|03301526
genistein|03311112
genistin|03311762
soybeans|03301075
soy nuts|00002852
soy powder|00002837
soy milk|03310351
tofu|03302496
miso|00002098
natto|00002196
soy sauce|03303751
fruit|03401105
vegetables|03401323
leguminosae|00001229
dried beans

### Data preprocessing chatgpt-4 SA_SNOMEDCT

In [85]:
cafeteria_SA_snomedct_results = pd.read_csv("results_chatgpt-4/food/CafeteriaSA_snomedct_response_GPT4.csv",delimiter=',')
cafeteria_SA_snomedct = pd.read_csv("CafeteriaCorpus/CafeteriaSA/CafeteriaSA_SNOMEDCT_reformatted_annotations.csv",delimiter='|')
first_column_cafeteria_SA_snomedct = set(cafeteria_SA_snomedct.iloc[:,0])

#print(article_results_chatGPT)

# we prepare the chatGPT data so that we can easily parse it
dictionary_of_chatGPT_results = {}
dictionary_of_chatGPT_results_without_MESH = {}
for index,row in cafeteria_SA_snomedct_results.iterrows():
    chatGPT_annotations = set()
    chatGPT_annotations_2 = set()
    all_annotations = row["result"]
    for chatGPTanno in str(all_annotations).split('\n'):
        if len(chatGPTanno) > 0 and chatGPTanno != 'nan':
                print(chatGPTanno)
                chatGPT_annotations.add((chatGPTanno.strip().split('|')[0],chatGPTanno.strip().split('|')[1]))
                chatGPT_annotations_2.add(chatGPTanno.strip().split('|')[0])
    dictionary_of_chatGPT_results[str(row[0])] = chatGPT_annotations
    dictionary_of_chatGPT_results_without_MESH[str(row[0])] = chatGPT_annotations_2
# test set
correct_entity = 0
incorrect_entity = 0
correct_entity_and_MESH = 0
incorrect_entity_and_MESH = 0
all_correct_entities = 0


### ------------------------------------------------------------------------------------------------------------------------

# cafeteria_SA_foodon
dictionary_of_results = {}
dictionary_of_results_without_MESH = {}
for article_identifier in first_column_cafeteria_SA_snomedct:
    filtered_cafeteria_SA_foodon = cafeteria_SA_snomedct[cafeteria_SA_snomedct.iloc[:, 0] == article_identifier]
    set_for_both = {}
    set_for_names = set()
    for index,row in filtered_cafeteria_SA_foodon.iterrows():
        name = str(row[1])
        identifiers = str(row[2]).split(';')
        list_of_possible_identifiers = set()
        for identity in identifiers:
            divide_identifiers = identity.split('/')
            get_correct = divide_identifiers[len(divide_identifiers)-1]
            list_of_possible_identifiers.add(get_correct)
        # 0 is the actually name and 1 is a set of possible identifiers
        set_for_both[name] = list_of_possible_identifiers
        set_for_names.add(name)
    dictionary_of_results[article_identifier] = set_for_both
    dictionary_of_results_without_MESH[article_identifier] = set_for_names
    all_correct_entities += len(set_for_names)
print(len(dictionary_of_chatGPT_results_without_MESH))
print(len(dictionary_of_results_without_MESH))

print(len(dictionary_of_chatGPT_results))
print(len(dictionary_of_results))


# Some of them are missing so we need to add them with empty array:
what_is_missing = []
for element in dictionary_of_chatGPT_results_without_MESH:
    does_it_exist = False
    for element2 in dictionary_of_results_without_MESH:
        if str(element) == str(element2):
            does_it_exist = True
    if not does_it_exist:
         what_is_missing.append(element)

         
for element in what_is_missing:
    dictionary_of_results[element] = {}
    dictionary_of_results_without_MESH[element] = set()

print(len(dictionary_of_chatGPT_results))
print(len(dictionary_of_results))


path_to_results_file_without_mesh = "basic_occurences_results/chatgpt-4/food/chatGPT_food_statistics_without_SNOMEDCT_SA_SNOMEDCT.txt"
path_to_results_file_with_mesh = "basic_occurences_results/chatgpt-4/food/chatGPT_food_statistics_with_SNOMEDCT_SA_SNOMEDCT.txt"
path_to_confusion_matrix_pure = "confusion_matrix_results_CHATGPT/chatgpt-4/food/confusion_matrix_food_pure_entities_SA_SNOMEDCT.csv"
path_to_confusion_matrix_partial_lapover = "confusion_matrix_results_CHATGPT/chatgpt-4/food/confusion_matrix_food_entities_test_contains_SA_SNOMEDCT.csv"
path_to_confusion_matrix_partial_full_lapover = "confusion_matrix_results_CHATGPT/chatgpt-4/food/confusion_matrix_food_entities_test_contains_and_chatgpt_contains_SA_SNOMEDCT.csv"


gluten-free products|77580000
celiac disease|609096000
selenium|20930005
grain|73239005
corn|317898006
rice|53003006
buckwheat|418641006
oat|20085008
amaranth|38298003
teff|162045409
quinoa|71112000
cheese|14160005
milk|76247003
milk|737532008
calcium|255582007
protein|87628006
phosphorus|259285001
hydroxyproline|46245007
creatine|76130009
tea|397430003
coffee|129313001
chocolate|450095007
cocoa|234592001
cinnamon|124953208
grape|234600002
pomegranate|41475000
red wine|329141000119100
berries|70376001
olive oil|425506004
red meat|9078004
processed meat|78201009
ultra-processed foods|762379003
dairy products|762721003
alcohol|404640003
whole grain cereals|762705008
nuts|412445004
vegetables|410942007
fruits|729528004
Curcuma longa|918633007
Nigella sativa|None
Zingiber officinale|387497006
Allium sativum|387495009
Crocus sativus|None
Piper nigrum|387487004
Capsicum annum|387494007
genistein|387231009
genistin|None
soybeans|36534000
soy nuts|None
soy powder|None
soy milk|260791009
tofu|6

### Data preprocessing chatgpt-4 FCD_FOODON

In [102]:
cafeteria_FCD_foodon_results = pd.read_csv("results_chatgpt-4/food/CafeteriaFCD_foodon_response_GPT4.csv",delimiter=',')
cafeteria_FCD_foodon = pd.read_csv("CafeteriaCorpus/CafeteriaFCD/CafeteriaFCD_foodon_reformatted_annotations.csv",delimiter='|')
first_column_cafeteria_FCD_foodon = set(cafeteria_FCD_foodon.iloc[:,0])

#print(article_results_chatGPT)

# we prepare the chatGPT data so that we can easily parse it
dictionary_of_chatGPT_results = {}
dictionary_of_chatGPT_results_without_MESH = {}
for index,row in cafeteria_FCD_foodon_results.iterrows():
    chatGPT_annotations = set()
    chatGPT_annotations_2 = set()
    all_annotations = row["result"]
    for chatGPTanno in str(all_annotations).split('\n'):
        if len(chatGPTanno) > 0 and str(chatGPTanno) != 'nan':
                print(chatGPTanno)
                chatGPT_annotations.add((chatGPTanno.strip().split('|')[0],chatGPTanno.strip().split('|')[1]))
                chatGPT_annotations_2.add(chatGPTanno.strip().split('|')[0])
    dictionary_of_chatGPT_results[str(row[0])] = chatGPT_annotations
    dictionary_of_chatGPT_results_without_MESH[str(row[0])] = chatGPT_annotations_2
# test set
correct_entity = 0
incorrect_entity = 0
correct_entity_and_MESH = 0
incorrect_entity_and_MESH = 0
all_correct_entities = 0


### ------------------------------------------------------------------------------------------------------------------------

# cafeteria_SA_foodon
dictionary_of_results = {}
dictionary_of_results_without_MESH = {}
for article_identifier in first_column_cafeteria_FCD_foodon:
    filtered_cafeteria_SA_foodon = cafeteria_FCD_foodon[cafeteria_FCD_foodon.iloc[:, 0] == article_identifier]
    set_for_both = {}
    set_for_names = set()
    for index,row in filtered_cafeteria_SA_foodon.iterrows():
        name = str(row[1])
        identifiers = str(row[2]).split(';')
        list_of_possible_identifiers = set()
        for identity in identifiers:
            list_of_possible_identifiers.add(identity.split('_')[1])
        # 0 is the actually name and 1 is a set of possible identifiers
        set_for_both[name] = list_of_possible_identifiers
        set_for_names.add(name)
    dictionary_of_results[article_identifier] = set_for_both
    dictionary_of_results_without_MESH[article_identifier] = set_for_names
    all_correct_entities += len(set_for_names)

print(len(dictionary_of_chatGPT_results_without_MESH))
print(len(dictionary_of_results_without_MESH))

print(len(dictionary_of_chatGPT_results))
print(len(dictionary_of_results))


# Some of them are missing so we need to add them with empty array:
what_is_missing = []
for element in dictionary_of_chatGPT_results_without_MESH:
    does_it_exist = False
    for element2 in dictionary_of_results_without_MESH:
        if str(element) == str(element2):
            does_it_exist = True
    if not does_it_exist:
         what_is_missing.append(element)

         
for element in what_is_missing:
    dictionary_of_results[element] = {}
    dictionary_of_results_without_MESH[element] = set()

print(len(dictionary_of_chatGPT_results))
print(len(dictionary_of_results))


path_to_results_file_without_mesh = "basic_occurences_results/chatgpt-4/food/chatGPT_food_statistics_without_FOODON_FCD_FOODON.txt"
path_to_results_file_with_mesh = "basic_occurences_results/chatgpt-4/food/chatGPT_food_statistics_with_FOODON_FCD_FOODON.txt"
path_to_confusion_matrix_pure = "confusion_matrix_results_CHATGPT/chatgpt-4/food/confusion_matrix_food_pure_entities_FCD_FOODON.csv"
path_to_confusion_matrix_partial_lapover = "confusion_matrix_results_CHATGPT/chatgpt-4/food/confusion_matrix_food_entities_test_contains_FCD_FOODON.csv"
path_to_confusion_matrix_partial_full_lapover = "confusion_matrix_results_CHATGPT/chatgpt-4/food/confusion_matrix_food_entities_test_contains_and_chatgpt_contains_FCD_FOODON.csv"


cream cheese|03307341
beef|03302089
olives|03301001
onion|03301768
Worcestershire sauce|03302402
walnuts|03303488
artichoke hearts|03411217
mozzarella cheese|03302068
parmesan cheese|03302782
mayonnaise|03411124
almonds|03302240
tortilla chips|03310956
crackers|03311584
sun-dried tomatoes|03401055
cream cheese|03411559
garlic|03411151
provolone cheese|03302040
pesto|03307367
garlic|03401203
salt|03302036
garbanzo beans|03411156
tahini|03361793
lemon juice|03472238
honey|03361751
water|03302759
olive oil|03471069
pita breads|03361706
rosemary|03461474
hummus|03337569
chicken|03302005
picante sauce|00001191
cumin|00001217
vegetable oil|00001587
corn tortilla|00001885
cheese|00002484
cream cheese|03302109
meat|03310719
pickle|03411103
ranch salad dressing mix|00001590
mayonnaise|00002123
milk|03420290
cream cheese|03414712
Cheddar cheese|03302036
almonds|03302449
rice|03411106
cold water|03402702
ground lamb|03307342
allspice|03414697
salt|00002186
pepper|03302051
garlic|03302761
lemon ju

### Data preprocessing chatgpt-4 FCD_SNOMEDCT

In [3]:
cafeteria_FCD_snomedct_results = pd.read_csv("results_chatgpt-4/food/CafeteriaFCD_snomedct_response_GPT4.csv",delimiter=',')
cafeteria_FCD_snomedct = pd.read_csv("CafeteriaCorpus/CafeteriaFCD/CafeteriaFCD_snomedct_reformatted_annotations.csv",delimiter='|')
first_column_cafeteria_FCD_snomedct = set(cafeteria_FCD_snomedct.iloc[:,0])
#print(article_results_chatGPT)

# we prepare the chatGPT data so that we can easily parse it
dictionary_of_chatGPT_results = {}
dictionary_of_chatGPT_results_without_MESH = {}
for index,row in cafeteria_FCD_snomedct_results.iterrows():
    chatGPT_annotations = set()
    chatGPT_annotations_2 = set()
    all_annotations = row["result"]
    for chatGPTanno in str(all_annotations).split('\n'):
        if len(chatGPTanno) > 0 and chatGPTanno != 'nan':
                print(chatGPTanno)
                chatGPT_annotations.add((chatGPTanno.strip().split('|')[0],chatGPTanno.strip().split('|')[1]))
                chatGPT_annotations_2.add(chatGPTanno.strip().split('|')[0])
    dictionary_of_chatGPT_results[str(row[0])] = chatGPT_annotations
    dictionary_of_chatGPT_results_without_MESH[str(row[0])] = chatGPT_annotations_2
# test set
correct_entity = 0
incorrect_entity = 0
correct_entity_and_MESH = 0
incorrect_entity_and_MESH = 0
all_correct_entities = 0


### ------------------------------------------------------------------------------------------------------------------------

# cafeteria_SA_foodon
dictionary_of_results = {}
dictionary_of_results_without_MESH = {}
for article_identifier in first_column_cafeteria_FCD_snomedct:
    filtered_cafeteria_SA_foodon = cafeteria_FCD_snomedct[cafeteria_FCD_snomedct.iloc[:, 0] == article_identifier]
    set_for_both = {}
    set_for_names = set()
    for index,row in filtered_cafeteria_SA_foodon.iterrows():
        name = str(row[1])
        identifiers = str(row[2]).split(';')
        list_of_possible_identifiers = set()
        for identity in identifiers:
            divide_identifiers = identity.split('/')
            get_correct = divide_identifiers[len(divide_identifiers)-1]
            list_of_possible_identifiers.add(get_correct)
        # 0 is the actually name and 1 is a set of possible identifiers
        set_for_both[name] = list_of_possible_identifiers
        set_for_names.add(name)
    dictionary_of_results[article_identifier] = set_for_both
    dictionary_of_results_without_MESH[article_identifier] = set_for_names
    all_correct_entities += len(set_for_names)

print(len(dictionary_of_chatGPT_results_without_MESH))
print(len(dictionary_of_results_without_MESH))

print(len(dictionary_of_chatGPT_results))
print(len(dictionary_of_results))


# Some of them are missing so we need to add them with empty array:
what_is_missing = []
for element in dictionary_of_chatGPT_results_without_MESH:
    does_it_exist = False
    for element2 in dictionary_of_results_without_MESH:
        if str(element) == str(element2):
            does_it_exist = True
    if not does_it_exist:
         what_is_missing.append(element)

         
for element in what_is_missing:
    dictionary_of_results[element] = {}
    dictionary_of_results_without_MESH[element] = set()

print(len(dictionary_of_chatGPT_results))
print(len(dictionary_of_results))


path_to_results_file_without_mesh = "basic_occurences_results/chatgpt-4/food/chatGPT_food_statistics_without_SNOMEDCT_FCD_SNOMEDCT.txt"
path_to_results_file_with_mesh = "basic_occurences_results/chatgpt-4/food/chatGPT_food_statistics_with_SNOMEDCT_FCD_SNOMEDCT.txt"
path_to_confusion_matrix_pure = "confusion_matrix_results_CHATGPT/chatgpt-4/food/confusion_matrix_food_pure_entities_FCD_SNOMEDCT.csv"
path_to_confusion_matrix_partial_lapover = "confusion_matrix_results_CHATGPT/chatgpt-4/food/confusion_matrix_food_entities_test_contains_FCD_SNOMEDCT.csv"
path_to_confusion_matrix_partial_full_lapover = "confusion_matrix_results_CHATGPT/chatgpt-4/food/confusion_matrix_food_entities_test_contains_and_chatgpt_contains_FCD_SNOMEDCT.csv"


cream cheese|703371004
beef|407935006
olives|247223001
onion|247197005
Worcestershire sauce|420922006
walnuts|262244000
artichoke hearts|260179009
mozzarella cheese|762642009
parmesan cheese|770491006
mayonnaise|70026000
almonds|226216000
tortilla chips|770634005
crackers|767999000
water|77417006
sun-dried tomatoes|760496005
cream cheese|729491003
garlic|736267005
provolone cheese|729418008
pesto|760258006
garlic|410297008
salt|87678004
garbanzo beans|405472008
tahini|721397006
lemon juice|733732004
honey|362399009
olive oil|40463003
pita breads|766887005
rosemary|72653000
hummus|698577002
chicken|735424
picante sauce|770544004
cumin|789474008
vegetable oil|723491000119105
corn tortilla|762711002
cheese|255602003
Cream Cheese|91934008
Meat|86093009
Pickle|109444004
ranch salad dressing mix|726227004
mayonnaise|726151000
milk|260297002
cream cheese|292073004
Cheddar cheese|228333003
almonds|260310006
rice|86084001
ground lamb|768141000000101
allspice|161581007
salt|41046007
pepper|40097

# Analysis of the data (Only entities)
_______

### PURE CORRECT ENTITIES

In [4]:
# analysis variables
pure_correct_entity = 0
incorrect_entity = 0

#print(dictionary_of_results)
#print(dictionary_of_chatGPT_results_without_MESH)
# First we check how many entities were correct

# Pure correct
for key in dictionary_of_results_without_MESH:
    test_results = dictionary_of_results_without_MESH[key]
    chatGPT_results = dictionary_of_chatGPT_results_without_MESH[str(key)]
    length_of_test = len(chatGPT_results)
    counter = 0
    for test in test_results:
        for actual in chatGPT_results:
            if actual.strip().upper() == test.upper():
                pure_correct_entity += 1
                counter += 1
                break
                
    incorrect_entity += length_of_test - counter

# Pure correct Entities
print("Pure correct entities: "+str(pure_correct_entity))
print("All correct entities: "+str(all_correct_entities))
print("Percentage(%): "+str((pure_correct_entity/all_correct_entities)*100)[0:5])
print("Number of excess entities (incorrect): "+str(incorrect_entity))

file1 = open(path_to_results_file_without_mesh, "a")
file1.write('*** PURE ENTITIES ***\n\nPure correct entities: '+str(pure_correct_entity)+'\nAll correct entities: '+str(all_correct_entities)+'\nPercentage(%): '+str((pure_correct_entity/all_correct_entities)*100)[0:5]+'\nNumber of excess entities (incorrect): '+str(incorrect_entity)+'\n\n')
file1.close()

Pure correct entities: 3916
All correct entities: 5519
Percentage(%): 70.95
Number of excess entities (incorrect): 4486


### If the test entity is contained in the CHATGPT entity

In [5]:
# analysis variables
correct_entity_with_test_contain = 0
incorrect_entity_with_test_contain = 0

#print(dictionary_of_results)
#print(dictionary_of_chatGPT_results_without_MESH)
# First we check how many entities were correct

# correct Entities with test contained in CHATGPT response for loop
for key in dictionary_of_results_without_MESH:
    test_results = dictionary_of_results_without_MESH[key]
    chatGPT_results = dictionary_of_chatGPT_results_without_MESH[str(key)]
    length_of_test = len(chatGPT_results)
    counter = 0
    for test in test_results:
        for actual in chatGPT_results:
            if actual.strip().upper() == test.upper() or test.upper() in actual.upper():
                correct_entity_with_test_contain += 1
                counter += 1
                break
                
    incorrect_entity_with_test_contain += length_of_test - counter

# correct Entities with test contained in CHATGPT response analysis
print("correct entities with test contain: "+str(correct_entity_with_test_contain))
print("All correct entities: "+str(all_correct_entities))
print("Percentage(%): "+str((correct_entity_with_test_contain/all_correct_entities)*100)[0:5])
print("Number of excess entities (incorrect): "+str(incorrect_entity_with_test_contain))

file1 = open(path_to_results_file_without_mesh, "a")
file1.write('*** ENTITIES from test are contained in chatGPT response ***\n\ncorrect entities: '+str(correct_entity_with_test_contain)+'\nAll correct entities: '+str(all_correct_entities)+'\nPercentage(%): '+str((correct_entity_with_test_contain/all_correct_entities)*100)[0:5]+'\nNumber of excess entities (incorrect): '+str(incorrect_entity_with_test_contain)+'\n\n')
file1.close()

correct entities with test contain: 5218
All correct entities: 5519
Percentage(%): 94.54
Number of excess entities (incorrect): 3184


### If test entities can be found in CHATGPTS response and vice versa

In [6]:
# analysis variables
correct_entity_with_test_contain_and_vice_versa = 0
incorrect_entity_with_test_contain_and_vice_versa = 0

#print(dictionary_of_results)
#print(dictionary_of_chatGPT_results_without_MESH)
# First we check how many entities were correct

# correct Entities with test contained in CHATGPT response for loop
for key in dictionary_of_results_without_MESH:
    test_results = dictionary_of_results_without_MESH[key]
    chatGPT_results = dictionary_of_chatGPT_results_without_MESH[str(key)]
    length_of_test = len(chatGPT_results)
    counter = 0
    for test in test_results:
        for actual in chatGPT_results:
            if actual.strip().upper() == test.upper() or test.upper() in actual.upper() or actual.upper() in test.upper():
                correct_entity_with_test_contain_and_vice_versa += 1
                counter += 1
                break
                
    incorrect_entity_with_test_contain_and_vice_versa += length_of_test - counter

# correct Entities with test contained in CHATGPT response analysis
print("correct entities with test contain and vice versa: "+str(correct_entity_with_test_contain_and_vice_versa))
print("All correct entities: "+str(all_correct_entities))
print("Percentage(%): "+str((correct_entity_with_test_contain_and_vice_versa/all_correct_entities)*100)[0:5])
print("Number of excess entities (incorrect): "+str(incorrect_entity_with_test_contain_and_vice_versa))

file1 = open(path_to_results_file_without_mesh, "a")
file1.write('*** ENTITIES from test are contained in chatGPT response and vice versa ***\n\ncorrect entities: '+str(correct_entity_with_test_contain_and_vice_versa)+'\nAll correct entities: '+str(all_correct_entities)+'\nPercentage(%): '+str((correct_entity_with_test_contain_and_vice_versa/all_correct_entities)*100)[0:5]+'\nNumber of excess entities (incorrect): '+str(incorrect_entity_with_test_contain_and_vice_versa)+'\n\n')
file1.close()

correct entities with test contain and vice versa: 5245
All correct entities: 5519
Percentage(%): 95.03
Number of excess entities (incorrect): 3157


# Analysis of data (with Identifier)
____

## Pure correct entities

In [7]:
# analysis variables
pure_correct_entity = 0
incorrect_entity = 0

#print(dictionary_of_results)
#print(dictionary_of_chatGPT_results_without_MESH)
# First we check how many entities were correct

# Pure correct
for key in dictionary_of_results_without_MESH:
    test_results = dictionary_of_results[key]
    chatGPT_results = dictionary_of_chatGPT_results[str(key)]
    length_of_test = len(chatGPT_results)
    counter = 0
    for test in test_results:
        for actual in chatGPT_results:
            found_correct = False
            for element in dictionary_of_results[key][test]:
                if actual[0].strip().upper() == test.upper() and actual[1] == element:
                    pure_correct_entity += 1
                    counter += 1
                    found_correct = True
                    break
            if found_correct:
                break

    incorrect_entity += length_of_test - counter

# Pure correct Entities
print("Pure correct entities: "+str(pure_correct_entity))
print("All correct entities: "+str(all_correct_entities))
print("Percentage(%): "+str((pure_correct_entity/all_correct_entities)*100)[0:5])
print("Number of excess entities (incorrect): "+str(incorrect_entity))

file1 = open(path_to_results_file_with_mesh, "a")
file1.write('*** PURE ENTITIES ***\n\nPure correct entities: '+str(pure_correct_entity)+'\nAll correct entities: '+str(all_correct_entities)+'\nPercentage(%): '+str((pure_correct_entity/all_correct_entities)*100)[0:5]+'\nNumber of excess entities (incorrect): '+str(incorrect_entity)+'\n\n')
file1.close()

Pure correct entities: 0
All correct entities: 5519
Percentage(%): 0.0
Number of excess entities (incorrect): 8402


### If the test entity is contained in the CHATGPT entity

In [8]:
# analysis variables
correct_entity_with_test_contain = 0
incorrect_entity_with_test_contain = 0

#print(dictionary_of_results)
#print(dictionary_of_chatGPT_results_without_MESH)
# First we check how many entities were correct

# correct Entities with test contained in CHATGPT response for loop
for key in dictionary_of_results_without_MESH:
    test_results = dictionary_of_results[key]
    chatGPT_results = dictionary_of_chatGPT_results[str(key)]
    length_of_test = len(chatGPT_results)
    counter = 0
    for test in test_results:
        for actual in chatGPT_results:
            found_correct = False
            for element in dictionary_of_results[key][test]:
                if (actual[0].strip().upper() == test.upper() or test.upper() in actual[0].strip().upper()) and (actual[1] == element):
                    correct_entity_with_test_contain += 1
                    counter += 1
                    found_correct = True
                    break
            if found_correct:
                break
    incorrect_entity_with_test_contain += length_of_test - counter

# correct Entities with test contained in CHATGPT response analysis
print("correct entities with test contain: "+str(correct_entity_with_test_contain))
print("All correct entities: "+str(all_correct_entities))
print("Percentage(%): "+str((correct_entity_with_test_contain/all_correct_entities)*100)[0:5])
print("Number of excess entities (incorrect): "+str(incorrect_entity_with_test_contain))

file1 = open(path_to_results_file_with_mesh, "a")
file1.write('*** ENTITIES from test are contained in chatGPT response ***\n\ncorrect entities: '+str(correct_entity_with_test_contain)+'\nAll correct entities: '+str(all_correct_entities)+'\nPercentage(%): '+str((correct_entity_with_test_contain/all_correct_entities)*100)[0:5]+'\nNumber of excess entities (incorrect): '+str(incorrect_entity_with_test_contain)+'\n\n')
file1.close()

correct entities with test contain: 0
All correct entities: 5519
Percentage(%): 0.0
Number of excess entities (incorrect): 8402


### If test entities can be found in CHATGPTS response and vice versa

In [9]:
# analysis variables
correct_entity_with_test_contain_and_vice_versa = 0
incorrect_entity_with_test_contain_and_vice_versa = 0

#print(dictionary_of_results)
#print(dictionary_of_chatGPT_results_without_MESH)
# First we check how many entities were correct

# correct Entities with test contained in CHATGPT response for loop
for key in dictionary_of_results_without_MESH:
    test_results = dictionary_of_results[key]
    chatGPT_results = dictionary_of_chatGPT_results[str(key)]
    length_of_test = len(chatGPT_results)
    counter = 0
    for test in test_results:
        for actual in chatGPT_results:
            found_correct = False
            for element in dictionary_of_results[key][test]:
                if (actual[0].strip().upper() == test.upper() or test.upper() in actual[0].upper() or actual[0].upper() in test.upper()) and (actual[1] == element):
                    correct_entity_with_test_contain_and_vice_versa += 1
                    counter += 1
                    found_correct = True
                    break
            if found_correct:
                break
            
    incorrect_entity_with_test_contain_and_vice_versa += length_of_test - counter

# correct Entities with test contained in CHATGPT response analysis
print("correct entities with test contain and vice versa: "+str(correct_entity_with_test_contain_and_vice_versa))
print("All correct entities: "+str(all_correct_entities))
print("Percentage(%): "+str((correct_entity_with_test_contain_and_vice_versa/all_correct_entities)*100)[0:5])
print("Number of excess entities (incorrect): "+str(incorrect_entity_with_test_contain_and_vice_versa))

file1 = open(path_to_results_file_with_mesh, "a")
file1.write('*** ENTITIES from test are contained in chatGPT response and vice versa ***\n\ncorrect entities: '+str(correct_entity_with_test_contain_and_vice_versa)+'\nAll correct entities: '+str(all_correct_entities)+'\nPercentage(%): '+str((correct_entity_with_test_contain_and_vice_versa/all_correct_entities)*100)[0:5]+'\nNumber of excess entities (incorrect): '+str(incorrect_entity_with_test_contain_and_vice_versa)+'\n\n')
file1.close()

correct entities with test contain and vice versa: 0
All correct entities: 5519
Percentage(%): 0.0
Number of excess entities (incorrect): 8402


# Analysis confusion matrix
_____

In [10]:
### Create correct dictionaries
pairs_test_set = {}
pairs_chatGPT_set = []
unique_chatGPT_ids = set()
unique_test_set = set()

# get all meshes from CHATGPT and their entities
for key in dictionary_of_chatGPT_results:
    for mesh_id in dictionary_of_chatGPT_results[key]:
        pairs_chatGPT_set.append((mesh_id[0],mesh_id[1]))
        unique_chatGPT_ids.add(mesh_id[1])

# get all unique meshes from test set
for key in dictionary_of_results:
    for mesh_id in dictionary_of_results[key]:
        pairs_test_set[mesh_id] = dictionary_of_results[key][mesh_id]
        for element in dictionary_of_results[key][mesh_id]:
            unique_test_set.add(element)

already_added = []
list_of_test_meshes = pairs_test_set
unique_test_set = list(unique_test_set)
list_of_chatGPT_MESHES = list(unique_chatGPT_ids)

### Function to create dataframe

In [11]:
def createRowForDataframe(key,array,all_chatGPT_unique_ids):
    array_of_values = {}
    array_of_values["test_FOODON_ID"] = key 
    for element in all_chatGPT_unique_ids:
        counter = 0
        for ground_base_id in array:
            if element == ground_base_id:
                counter += 1
        array_of_values[element] = counter
    return array_of_values

## Pure correct entities

In [12]:
dictionary_of_matrix_occurences = {}
# Now we need to loop through all CHATGPT responses with each test mesh
alternatives = ["a","b","c","d","e"]
for test_entity in list_of_test_meshes:
    array_of_mesh_ids = []
    for element in list_of_test_meshes[test_entity]:
        for entity_set in pairs_chatGPT_set:
            if test_entity.upper() == entity_set[0].upper():
                array_of_mesh_ids.append(entity_set[1])
        if element in dictionary_of_matrix_occurences:
            for el in alternatives:
                new_element = str(element)+str(el)
                if new_element not in dictionary_of_matrix_occurences:
                    break
            dictionary_of_matrix_occurences[new_element] = array_of_mesh_ids
        else:        
            dictionary_of_matrix_occurences[element] = array_of_mesh_ids

confusion_matrix = pd.DataFrame(columns=['test_FOODON_ID']+list_of_chatGPT_MESHES)
# now we create rows for dataFrames
for keys in dictionary_of_matrix_occurences:
    confusion_matrix = confusion_matrix.append(createRowForDataframe(keys,dictionary_of_matrix_occurences[keys],list_of_chatGPT_MESHES),ignore_index=True)


C:\Users\matev\AppData\Local\Temp\ipykernel_4324\681188757.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  confusion_matrix = confusion_matrix.append(createRowForDataframe(keys,dictionary_of_matrix_occurences[keys],list_of_chatGPT_MESHES),ignore_index=True)
C:\Users\matev\AppData\Local\Temp\ipykernel_4324\681188757.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  confusion_matrix = confusion_matrix.append(createRowForDataframe(keys,dictionary_of_matrix_occurences[keys],list_of_chatGPT_MESHES),ignore_index=True)
C:\Users\matev\AppData\Local\Temp\ipykernel_4324\681188757.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  confusion_matrix = confusion_matrix.append(createRowForDataframe(keys,dictionary_of_matrix_occu

#### Clean confusion matrix and save it into a csv

In [13]:
# Check if each column contains only zeros
zero_columns = confusion_matrix.columns[confusion_matrix.eq(0).all()]
# Drop the zero columns from the dataframe
confusion_matrix = confusion_matrix.drop(zero_columns, axis=1)

# Get the subset of columns excluding the 'ID' column
columns_to_check = confusion_matrix.columns[~(confusion_matrix.columns == 'test_FOODON_ID')]

# Check if each row (excluding the 'ID' column) contains only zeros
zero_rows = confusion_matrix.loc[(confusion_matrix[columns_to_check] == 0).all(axis=1)]

# Drop the zero rows from the dataframe
confusion_matrix = confusion_matrix.drop(zero_rows.index)

confusion_matrix.to_csv(path_to_confusion_matrix_pure,index=False)

## If the test entity is contained in the CHATGPT entity

In [14]:
dictionary_of_matrix_occurences = {}
# Now we need to loop through all CHATGPT responses with each test mesh

alternatives = ["a","b","c","d","e"]
for test_entity in list_of_test_meshes:
    array_of_mesh_ids = []
    for element in list_of_test_meshes[test_entity]:
        for entity_set in pairs_chatGPT_set:
            if test_entity.upper() == entity_set[0].strip().upper() or test_entity.upper() in entity_set[0].strip().upper():
                array_of_mesh_ids.append(entity_set[1])
        
        if element in dictionary_of_matrix_occurences:
            for el in alternatives:
                new_element = str(element)+str(el)
                if new_element not in dictionary_of_matrix_occurences:
                    break
            dictionary_of_matrix_occurences[new_element] = array_of_mesh_ids
        else:        
            dictionary_of_matrix_occurences[element] = array_of_mesh_ids
            


confusion_matrix = pd.DataFrame(columns=['test_FOODON_ID']+list_of_chatGPT_MESHES)
# now we create rows for dataFrames
for keys in dictionary_of_matrix_occurences:
    confusion_matrix = confusion_matrix.append(createRowForDataframe(keys,dictionary_of_matrix_occurences[keys],list_of_chatGPT_MESHES),ignore_index=True)


C:\Users\matev\AppData\Local\Temp\ipykernel_4324\1374126061.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  confusion_matrix = confusion_matrix.append(createRowForDataframe(keys,dictionary_of_matrix_occurences[keys],list_of_chatGPT_MESHES),ignore_index=True)
C:\Users\matev\AppData\Local\Temp\ipykernel_4324\1374126061.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  confusion_matrix = confusion_matrix.append(createRowForDataframe(keys,dictionary_of_matrix_occurences[keys],list_of_chatGPT_MESHES),ignore_index=True)
C:\Users\matev\AppData\Local\Temp\ipykernel_4324\1374126061.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  confusion_matrix = confusion_matrix.append(createRowForDataframe(keys,dictionary_of_matrix_o

#### Clean confusion matrix and save it into a csv

In [15]:
# Check if each column contains only zeros
zero_columns = confusion_matrix.columns[confusion_matrix.eq(0).all()]
# Drop the zero columns from the dataframe
confusion_matrix = confusion_matrix.drop(zero_columns, axis=1)

# Get the subset of columns excluding the 'ID' column
columns_to_check = confusion_matrix.columns[~(confusion_matrix.columns == 'test_FOODON_ID')]

# Check if each row (excluding the 'ID' column) contains only zeros
zero_rows = confusion_matrix.loc[(confusion_matrix[columns_to_check] == 0).all(axis=1)]

# Drop the zero rows from the dataframe
confusion_matrix = confusion_matrix.drop(zero_rows.index)

confusion_matrix.to_csv(path_to_confusion_matrix_partial_lapover,index=False)

### If test entities can be found in CHATGPTS response and vice versa

In [16]:
dictionary_of_matrix_occurences = {}
# Now we need to loop through all CHATGPT responses with each test mesh
alternatives = ["a","b","c","d","e"]
for test_entity in list_of_test_meshes:
    array_of_mesh_ids = []
    for element in list_of_test_meshes[test_entity]:
        for entity_set in pairs_chatGPT_set:
            if test_entity.upper() == entity_set[0].strip().upper() or test_entity.upper() in entity_set[0].strip().upper() or entity_set[0].strip().upper() in test_entity.upper():
                array_of_mesh_ids.append(entity_set[1])
    
        if element in dictionary_of_matrix_occurences:
            for el in alternatives:
                new_element = str(element)+str(el)
                if new_element not in dictionary_of_matrix_occurences:
                    break
            dictionary_of_matrix_occurences[new_element] = array_of_mesh_ids
        else:        
            dictionary_of_matrix_occurences[element] = array_of_mesh_ids    


confusion_matrix = pd.DataFrame(columns=['test_FOODON_ID']+list_of_chatGPT_MESHES)
# now we create rows for dataFrames
for keys in dictionary_of_matrix_occurences:
    confusion_matrix = confusion_matrix.append(createRowForDataframe(keys,dictionary_of_matrix_occurences[keys],list_of_chatGPT_MESHES),ignore_index=True)


C:\Users\matev\AppData\Local\Temp\ipykernel_4324\4069418074.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  confusion_matrix = confusion_matrix.append(createRowForDataframe(keys,dictionary_of_matrix_occurences[keys],list_of_chatGPT_MESHES),ignore_index=True)
C:\Users\matev\AppData\Local\Temp\ipykernel_4324\4069418074.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  confusion_matrix = confusion_matrix.append(createRowForDataframe(keys,dictionary_of_matrix_occurences[keys],list_of_chatGPT_MESHES),ignore_index=True)
C:\Users\matev\AppData\Local\Temp\ipykernel_4324\4069418074.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  confusion_matrix = confusion_matrix.append(createRowForDataframe(keys,dictionary_of_matrix_o

#### Clean confusion matrix and save it into a csv

In [17]:
# Check if each column contains only zeros
zero_columns = confusion_matrix.columns[confusion_matrix.eq(0).all()]
# Drop the zero columns from the dataframe
confusion_matrix = confusion_matrix.drop(zero_columns, axis=1)

# Get the subset of columns excluding the 'ID' column
columns_to_check = confusion_matrix.columns[~(confusion_matrix.columns == 'test_MESH_ID')]

# Check if each row (excluding the 'ID' column) contains only zeros
zero_rows = confusion_matrix.loc[(confusion_matrix[columns_to_check] == 0).all(axis=1)]

# Drop the zero rows from the dataframe
confusion_matrix = confusion_matrix.drop(zero_rows.index)

confusion_matrix.to_csv(path_to_confusion_matrix_partial_full_lapover,index=False)